# Create Pangenome XML <a class="tocSkip">
12 Aug 2022; Julian 

**Code to create an XML to submit pangenomes as "analysis" objects to ENA. This notebook relies on a Google Sheet to have all of the neccesary information that will go into the XML. The Year1 data can be found [here](https://docs.google.com/spreadsheets/d/169OS-mKkR7qLzE5kzkHWob2bJckLhQbUuMxeH7i_upU/edit#gid=472507643).**
    
**Note that the Analysis XSD and instructions on uploading the XML can be found [here](https://ena-docs.readthedocs.io/en/latest/submit/analyses.html), but the XSD does not currently reflect the new Pangenone requirements. If needed, you can view the Pangenome metadata proposal [here](https://docs.google.com/document/d/1610m__E0yLPo3lkad2vOonH1AO6BCyjkR7qajc7pSVY/edit?usp=sharing).**

# Preparation

In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
import pandas as pd
import os

In [2]:
## Sample to write XML for (must match column names in Google sheets)
file_to_write = "hprc-v1.0-mc-grch38"

In [3]:
# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"


# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: AnVIL_HPRC_Data_Transfer
Workspace storage bucket: gs://fc-cba31066-5983-4306-b66e-bdcfb644fb32/


# Function Definitions

In [4]:
def create_assembly_el(assembly_acc: str):
    """
    returns element:
    <ASSEMBLY>
        <STANDARD accession=assembly_acc/>
    </ASSEMBLY>  
    """
    assembly_el   = ET.Element("ASSEMBLY")

    standard_el   = ET.Element("STANDARD")
    standard_el.set('accession', assembly_acc)
    
    assembly_el.append(standard_el)

    return assembly_el

In [5]:
def create_analysis_attribute_el(tag_str: str, value_str: str):
    """
    returns element:
    <ANALYSIS_ATTRIBUTE>
        <TAG>tag_str</TAG>
        <VALUE>value_str</VALUE>
    </ANALYSIS_ATTRIBUTE>  
    """
    analysis_attribute_el   = ET.Element("ANALYSIS_ATTRIBUTE")

    attribute_tag_el        = ET.Element("TAG")
    attribute_tag_el.text      = tag_str

    attribute_value_el      = ET.Element("VALUE")
    attribute_value_el.text = value_str

    analysis_attribute_el.append(attribute_tag_el)
    analysis_attribute_el.append(attribute_value_el)

    return analysis_attribute_el

# Create Analysis Element

## Pull In  Analysis Elements Info (Except SAMPLE_REF)

In [6]:
## Read in table with analysis info data
sheet_id = "169OS-mKkR7qLzE5kzkHWob2bJckLhQbUuMxeH7i_upU"
sheet_name = "analysis"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
analysis_df = pd.read_csv(url)

In [7]:
analysis_df.head()

,field name,hprc-v1.0-minigraph-chm13,hprc-v1.0-minigraph-grch38,hprc-v1.0-mc-chm13,hprc-v1.0-mc-grch38,hprc-v1.0-pggb
0,TITLE,HPRC Minigraph Pangenome (CHM13 based) v1.0,HPRC Minigraph Pangenome (GRCh38 based) v1.0,HPRC Minigraph-CACTUS Pangenome (CHM13 based) ...,HPRC Minigraph-CACTUS Pangenome (GRCh38 based)...,HPRC PGGB Pangenome v1.0
1,DESCRIPTION,Human Pangenome Reference Consortium (HPRC) pa...,Human Pangenome Reference Consortium (HPRC) pa...,Human Pangenome Reference Consortium (HPRC) pa...,Human Pangenome Reference Consortium (HPRC) pa...,Human Pangenome Reference Consortium (HPRC) pa...
2,STUDY_REF,PRJEB55385,PRJEB55386,PRJEB55387,PRJEB55388,PRJEB54791
3,filename,hprc-v1.0-minigraph-chm13.gfa.gz,hprc-v1.0-minigraph-grch38.gfa.gz,hprc-v1.0-mc-chm13.gfa.gz,hprc-v1.0-mc-grch38.gfa.gz,hprc-v1.0-pggb.gfa.gz
4,filetype,gfa,gfa,gfa,gfa,gfa


In [8]:
## Need to set index
analysis_df = analysis_df.set_index('field name')

## Create Analysis Elements (Except SAMPLE_REF)

In [9]:
## Just pull column for sample we are going to write
sample_analysis_df = analysis_df[file_to_write]

In [10]:
## Get values that we will write
alias           = file_to_write
title           = sample_analysis_df['TITLE']
descr           = sample_analysis_df['DESCRIPTION']
study_acc       = sample_analysis_df['STUDY_REF']
filename        = sample_analysis_df['filename']
filetype        = sample_analysis_df['filetype']
checksum_method = sample_analysis_df['checksum_method']
checksum        = sample_analysis_df['checksum']

In [11]:
## Create TITLE element
title_el      = ET.Element("TITLE")
title_el.text = title


## Create DESCRIPTION element
descr_el      = ET.Element("DESCRIPTION")
descr_el.text = descr


## Create STUDY_REF element
studyref_el      = ET.Element("STUDY_REF")
studyref_el.set('accession', study_acc)


## Create FILE & FILES elements (nest FILE inside FILES)
file_el  = ET.Element("FILE")
file_el.set('filename', filename)
file_el.set('filetype', filetype)
file_el.set('checksum', checksum)
file_el.set('checksum_method', checksum_method)

files_el = ET.Element("FILES")
files_el.append(file_el)

In [12]:
## Add sub elements to ANALYSIS element
## Note that we will add FILES element later (needs to go after SAMPLE_REF AND ASSEMBLY)
analysis_el = ET.Element("ANALYSIS")
analysis_el.set('alias', alias)

analysis_el.append(title_el)
analysis_el.append(descr_el)
analysis_el.append(studyref_el)

## Pull In SAMPLE_REF Info

In [13]:
## Read in table with BioSample IDs
sheet_id = "169OS-mKkR7qLzE5kzkHWob2bJckLhQbUuMxeH7i_upU"
sheet_name = "samples"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
sample_df = pd.read_csv(url)

In [14]:
sample_df.head()

,sample,biosample
0,HG00438,SAMN17861652
1,HG00621,SAMN17861653
2,HG00673,SAMN17861654
3,HG00735,SAMN17861655
4,HG00741,SAMN17861656


## Create SAMPLE_REF Elements

In [15]:
for index, row in sample_df.iterrows():

    biosample_id = row['biosample']

    sampleref_el      = ET.Element("SAMPLE_REF")
    sampleref_el.set('accession', biosample_id)
    
    analysis_el.append(sampleref_el)

## Pull in Assembly Accessions

In [16]:
## Read in table with BioSample IDs
sheet_id = "169OS-mKkR7qLzE5kzkHWob2bJckLhQbUuMxeH7i_upU"
sheet_name = "assemblies"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
assembly_df = pd.read_csv(url)

In [17]:
assembly_df.head()

,sample,accession,note
0,HG00438,GCA_018471515.1,maternal
1,HG00621,GCA_018472605.1,maternal
2,HG00673,GCA_018472565.1,maternal
3,HG00735,GCA_018472765.1,maternal
4,HG00741,GCA_018471095.1,maternal


## Create ANALYSIS_TYPE, ASSEMBLY_GRAPH, and ASSEMBLY Elements

In [18]:
## We are putting the pangenome under ASSEMBLY_GRAPH
pangenome_graph_el = ET.Element("ASSEMBLY_GRAPH")

In [19]:
for index, row in assembly_df.iterrows():

    accession = row['accession']

    assembly_el = create_assembly_el(accession)
    
    pangenome_graph_el.append(assembly_el)

In [20]:
analysis_type_el   = ET.Element("ANALYSIS_TYPE")
analysis_type_el.append(pangenome_graph_el)

In [21]:
## NOW we can add files element
analysis_el.append(analysis_type_el)
analysis_el.append(files_el)

# Create Analysis Attributes Elements

Create analysis attributes element

In [22]:
analysis_attributes_el = ET.Element("ANALYSIS_ATTRIBUTES")

## Add Analysis Attributes

In [23]:
## Read in table with analysis attributes
sheet_id = "169OS-mKkR7qLzE5kzkHWob2bJckLhQbUuMxeH7i_upU"
sheet_name = "analysis_attributes"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
anal_attr_df = pd.read_csv(url)

In [24]:
anal_attr_df.head()

,TAG,VALUE,Rules,example,Required,hprc-v1.0-minigraph-chm13,hprc-v1.0-minigraph-grch38,hprc-v1.0-mc-chm13,hprc-v1.0-mc-grch38,hprc-v1.0-pggb
0,number of nodes,"Number of segments, or S lines, in the GFA file.",numeric,"424,643",recommended,493631,424643,85591995,81751614,110884673
1,number of components,Connected components in the graph,numeric,25,recommended,25,194,2445,2624,25
2,number of edges,Number of edges/links (GFA L lines) in the pan...,numeric,"637,628",recommended,738529,637628,118409526,113258931,154756169
3,length (bp),Total number of nucleotides in the nodes/segme...,numeric,"3,239,764,787",recommended,3365688482,3239764787,3324657754,3287932785,8415267572
4,total path length (bp),Number of bases in all paths included in the g...,numeric,"3,239,764,787",recommended,NaN,NaN,257143252360,254821009311,271199380512


In [25]:
anal_attr_df = anal_attr_df.set_index('TAG')

## Loop through attributes and add ANALYSIS_ATTRIBUTEs

In [26]:
for index, row in anal_attr_df.iterrows():
    tag_name  = row.name
    tag_value = row[file_to_write]
    
    if not pd.isna(tag_value):

        analysis_attribute_el = create_analysis_attribute_el(tag_name, tag_value)
        analysis_attributes_el.append(analysis_attribute_el)

## Add (Common) Sample Names

In [27]:
## Read in table with common/Coriell sample names
sheet_id = "169OS-mKkR7qLzE5kzkHWob2bJckLhQbUuMxeH7i_upU"
sheet_name = "samples"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
sample_df = pd.read_csv(url)

In [28]:
sample_df.head()

,sample,biosample
0,HG00438,SAMN17861652
1,HG00621,SAMN17861653
2,HG00673,SAMN17861654
3,HG00735,SAMN17861655
4,HG00741,SAMN17861656


## Loop through samples and add as ANALYSIS_ATTRIBUTEs

In [29]:
for index, row in sample_df.iterrows():
    sample_name = row['sample']
    sample_name_el = create_analysis_attribute_el("sample name", sample_name)
    
    analysis_attributes_el.append(sample_name_el)

In [30]:
# Create Final Tree

In [31]:
## Old way
## Add ANALYSIS_ATTRIBUTES to ANALYSIS_SET (root element)
# analysis_set_el = ET.Element("ANALYSIS_SET")

# analysis_set_el.append(analysis_el)
# analysis_set_el.append(analysis_attributes_el)

In [32]:
## Add ANALYSIS_ATTRIBUTES to ANALYSIS_SET (root element)
analysis_set_el = ET.Element("ANALYSIS_SET")

analysis_el.append(analysis_attributes_el)
analysis_set_el.append(analysis_el)

# Write, Check, & Upload

## Write XML File

In [33]:
write_name = f"{file_to_write}.xml"

xmlstr = minidom.parseString(ET.tostring(analysis_set_el, encoding='UTF-8')).toprettyxml(indent="   ")
with open(write_name, "w") as f:
    f.write(xmlstr)

## Check results

In [34]:
! cat {write_name}

<?xml version="1.0" ?>
<ANALYSIS_SET>
   <ANALYSIS alias="hprc-v1.0-mc-grch38">
      <TITLE>HPRC Minigraph-CACTUS Pangenome (GRCh38 based) v1.0</TITLE>
      <DESCRIPTION>Human Pangenome Reference Consortium (HPRC) pangenome built with the minigraph-CACTUS pipeline from 88 year 1 assemblies plus GRCh38 (included as the reference assembly) and CHM13.</DESCRIPTION>
      <STUDY_REF accession="PRJEB55388"/>
      <SAMPLE_REF accession="SAMN17861652"/>
      <SAMPLE_REF accession="SAMN17861653"/>
      <SAMPLE_REF accession="SAMN17861654"/>
      <SAMPLE_REF accession="SAMN17861655"/>
      <SAMPLE_REF accession="SAMN17861656"/>
      <SAMPLE_REF accession="SAMN17861657"/>
      <SAMPLE_REF accession="SAMN17861658"/>
      <SAMPLE_REF accession="SAMN17861232"/>
      <SAMPLE_REF accession="SAMN17861659"/>
      <SAMPLE_REF accession="SAMN17861233"/>
      <SAMPLE_REF accession="SAMN17861234"/>
      <SAMPLE_REF accession="SAMN17861235"/>
      <SAMPLE_REF accession="SAMN17861236"/>
      

## Upload To Bucket

In [35]:
! gsutil cp {write_name} {bucket}Y1_Pangenome_XML/{write_name}

Copying file://hprc-v1.0-mc-grch38.xml [Content-Type=application/xml]...
/ [1 files][ 20.0 KiB/ 20.0 KiB]                                                
Operation completed over 1 objects/20.0 KiB.                                     


In [36]:
print(f"{bucket}Y1_Pangenome_XML/{write_name}")

gs://fc-cba31066-5983-4306-b66e-bdcfb644fb32/Y1_Pangenome_XML/hprc-v1.0-mc-grch38.xml
